Importing our dataset and exploring

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/OpinosisDataset1.0_0/topics

/content/gdrive/My Drive/OpinosisDataset1.0_0/topics


In [3]:
!ls

accuracy_garmin_nuvi_255W_gps.txt.data
bathroom_bestwestern_hotel_sfo.txt.data
battery-life_amazon_kindle.txt.data
battery-life_ipod_nano_8gb.txt.data
battery-life_netbook_1005ha.txt.data
buttons_amazon_kindle.txt.data
comfort_honda_accord_2008.txt.data
comfort_toyota_camry_2007.txt.data
directions_garmin_nuvi_255W_gps.txt.data
display_garmin_nuvi_255W_gps.txt.data
eyesight-issues_amazon_kindle.txt.data
features_windows7.txt.data
fonts_amazon_kindle.txt.data
food_holiday_inn_london.txt.data
food_swissotel_chicago.txt.data
free_bestwestern_hotel_sfo.txt.data
gas_mileage_toyota_camry_2007.txt.data
interior_honda_accord_2008.txt.data
interior_toyota_camry_2007.txt.data
keyboard_netbook_1005ha.txt.data
location_bestwestern_hotel_sfo.txt.data
location_holiday_inn_london.txt.data
mileage_honda_accord_2008.txt.data
navigation_amazon_kindle.txt.data
parking_bestwestern_hotel_sfo.txt.data
pegasus
performance_honda_accord_2008.txt.data
performance_netbook_1005ha.txt.data
price_amazon_kindle.txt.

In [3]:
import os
os.listdir()

['display_garmin_nuvi_255W_gps.txt.data',
 'gas_mileage_toyota_camry_2007.txt.data',
 'fonts_amazon_kindle.txt.data',
 'eyesight-issues_amazon_kindle.txt.data',
 'battery-life_amazon_kindle.txt.data',
 'accuracy_garmin_nuvi_255W_gps.txt.data',
 'battery-life_ipod_nano_8gb.txt.data',
 'bathroom_bestwestern_hotel_sfo.txt.data',
 'directions_garmin_nuvi_255W_gps.txt.data',
 'battery-life_netbook_1005ha.txt.data',
 'food_swissotel_chicago.txt.data',
 'interior_honda_accord_2008.txt.data',
 'free_bestwestern_hotel_sfo.txt.data',
 'location_bestwestern_hotel_sfo.txt.data',
 'interior_toyota_camry_2007.txt.data',
 'food_holiday_inn_london.txt.data',
 'buttons_amazon_kindle.txt.data',
 'comfort_toyota_camry_2007.txt.data',
 'keyboard_netbook_1005ha.txt.data',
 'features_windows7.txt.data',
 'comfort_honda_accord_2008.txt.data',
 'service_swissotel_hotel_chicago.txt.data',
 'performance_netbook_1005ha.txt.data',
 'mileage_honda_accord_2008.txt.data',
 'satellite_garmin_nuvi_255W_gps.txt.data',


Exploring our dataset and choosin one document for extractive text summarization

In [4]:
file = open("free_bestwestern_hotel_sfo.txt.data", "r", encoding='cp1252')
text = file.read()
file.close()

In [5]:
text

"The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n The service was good, very friendly staff and we loved the free wine reception each night .\n has free wireless and help you with transportation needs .\n The nightly free wine tasting from 5 ,  6 pm is a brilliant idea and gets guests together to socialise witheach other .\n They have a happy hour where you have a couple offree drinks between 5 and 6 and this gives you a chance to meet other travellers .\n free wifi and happy hour wine every afternoon !\n The parking was free, which was great, and the hotel was conveniently located for public transport, and local attractions .\n We were also told of the free wine reception at 5pm, grea

Extractive Summarization

In [6]:
!pip install bert-extractive-summarizer
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 8.5MB/s 
     |████████████████████████████████| 3.3MB 41.2MB/s 
     |████████████████████████████████| 901kB 49.9MB/s 
     |████████████████████████████████| 1.2MB 6.7MB/s 


In [7]:
from summarizer import Summarizer
model = Summarizer()


In [8]:
result = model(text, min_length=4)
summary = "".join(result)
print(summary)

The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room . free wifi and happy hour wine every afternoon ! Free internet access and Happy Hour every afternoon . The free wine tasting every night was a wonderful extra and welcomed relaxment in between a long day of sightseeing and evenings out . The hotel has one computer   available with free internet . The room also has free wireless which I've read other hotels nearby don't . I didn't get to take advantage of the free coffee in the morning, but my husband and I did go to the 5pm, 6pm cocktail hour, which we enjoyed . This is a hassle, free place to stay with a dog . The hotel had free coffee and drinks everyday and free access to wireless internet . The free wine beer and coffee hot chocolate biscotti is a wonderful touch and we took full advantage of it . The late afternoon free wine reception was a really nice touch and we enjoyed this very much . Alth

Abstractive Summarization

BART model


In [9]:
!pip install transformers

In [10]:
from transformers import pipeline
import os

In [11]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
summarizer = pipeline("summarization")

In [16]:
summary_text = summarizer(summary, max_length=150, min_length=100)[0]
print(summary_text)

[{'summary_text': " Free wine tasting every night was a wonderful extra and welcomed relaxment in between a long day of sightseeing and evenings out . The hotel had free coffee and drinks everyday and free access to wireless internet . The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room . I didn't get to take advantage of the free coffee in the morning, but my husband and I did go to the 5pm, 6pm cocktail hour, which we enjoyed . This is a hassle, free place to stay with a dog ."}]


T5 model

In [17]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')


In [19]:
preprocess_text = summary.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=40,
                                    max_length=150,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)


original text preprocessed: 
 The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room . free wifi and happy hour wine every afternoon ! Free internet access and Happy Hour every afternoon . The free wine tasting every night was a wonderful extra and welcomed relaxment in between a long day of sightseeing and evenings out . The hotel has one computer   available with free internet . The room also has free wireless which I've read other hotels nearby don't . I didn't get to take advantage of the free coffee in the morning, but my husband and I did go to the 5pm, 6pm cocktail hour, which we enjoyed . This is a hassle, free place to stay with a dog . The hotel had free coffee and drinks everyday and free access to wireless internet . The free wine beer and coffee hot chocolate biscotti is a wonderful touch and we took full advantage of it . The late afternoon free wine reception was a really nice touch and we

Evaluation Metrics- ROUGE

In [23]:
pip install rouge

In [24]:
from rouge import Rouge

In [25]:
rouge = Rouge()

In [26]:
model_out_BART = "Hotel had free coffee and drinks everyday and free access to wireless internet . Free wine tasting every night was a wonderful extra and welcomed relaxment in between a long day of sightseeing and evenings out . Hotel has one computer  available with free internet . The room also has free wireless which I've read other hotels nearby don't ."
reference = """Hotel offers free wine in the evening and coffee and biscotti in the morning.
Amenities includes free parking, free wifi, and free wireless."""

In [28]:
rouge.get_scores(model_out_BART, reference)

[{'rouge-1': {'f': 0.30379746422688675,
   'p': 0.21428571428571427,
   'r': 0.5217391304347826},
  'rouge-2': {'f': 0.07792207384044549,
   'p': 0.05454545454545454,
   'r': 0.13636363636363635},
  'rouge-l': {'f': 0.2295081928513841, 'p': 0.15555555555555556, 'r': 0.4375}}]

In [29]:
model_out_t5 = "the wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room. free wifi and happy hour wine every afternoon! Free internet access and Happy Hour every evening!!"
reference = """Hotel offers free wine in the evening and coffee and biscotti in the morning.
Amenities includes free parking, free wifi, and free wireless."""

In [30]:
rouge.get_scores(model_out_t5, reference)

[{'rouge-1': {'f': 0.27272726818640963,
   'p': 0.20930232558139536,
   'r': 0.391304347826087},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.12499999555555572, 'p': 0.09375, 'r': 0.1875}}]